# ENI Port Scan

Use this runbook to diagnose the source of a port scan on an AWS ENI

## Parameters
* **eni_id**: The ENI ID of the network adapters receiving the port scan.
* **account_id**: THe AWS account that the port scan is occurring.

In [1]:
from jupyterirtools import sso

account_id = "251344881676"
eni_id = "eni-04050d5ac94cd243b"

sso.login("Jupyter-IR-ViewOnly", account_id)

Credentials expire in 3 hours and 45 minutes

Login Successful, click to open [AWS Console](https://signin.aws.amazon.com/federation?Action=login&Issuer=Example.org&Destination=https%3A%2F%2Fconsole.aws.amazon.com%2F&SigninToken=snXhfy1dNLrtZ9NiVw-8rt-4dEDfWfxoRWwcNrFbrhiCpdGTFHWZxzy9g2Uibwvi4etQ4LCTNDIEaL7-ptivSHk2YTnpufM7ZLdQsAMvXF3vWVFb8YFLOq4oiZlDsAkkdpwLSilYNqwlQSbm2SJ2nPKYp2LjS9Fc6ZSQC08l_erRhQz4k27VyMLEShuxM9SDDDJBxVyVQNva4v_Y2Eg5k3pgUHlhhh7SH1X7tmDA2SkZ1X72rMsRZ8S2GRdq5sX-FhErV0xgOsJP4FelBjpFCVA-Z8jl79TJGOzBsFL3cZA5xBKmzmdx_Vw3aFT5kf5su3IVOE4h89bVXpdcmV1UGSeDnyZr9HKK5WdHipkZWO-Px5496V2ow5jBuzofs8MZLmiB4nOLX9j3nMkg5gz9Q8wdKt6rLVwbcbPxugC_dGST3l1MZS0TR0TQcaZFGnozoxlbwz2GjVACuPUdPGTB8PWKpwWLmoYc4gNyZDzMOp37-_H6P-0YOfth-Tm57564KsyL5O6zjxDTS46IKnFAvZvh8bLCfiep5cp1XX4iKb-R7b2aDXMg8O4BMNBl6JZ2_odsVsoBH3dGirjeB6hQ9drerfhe2ywp_-liIr_cZikaIT_gL3ZOlwAadMnVliQinU-nceIFcnM3WrFDmU-80TDiucrhQMozvH0Lmkjsgvj3r94aEc7flATIq3olqHkIrlZf24IaC_Pbvkzhop6ToVt4ql0jqUYbTFVen6O-r7oehc5G3AOv5sPKpH4Sk4MHHSaPk37C_TFbfZcb8aMdW-6eKkuZqqJLaoG38LToGaFMKAiEcWWTIJbbHwLmVM3Qyfo1s7IhxgiZIMEtA1Ek_xt8z9aLWkLjIwz6p2CIs24uduSENlG2F2u7Fz1lOLZzWLleicO-D6Sd-0kZyapYIPicDpH1T59wpmiBggqqOHvEGFYjn1aNJPabZ31VcnDHQI8bgYQ34v3wTqXimhCXYqRF9YS1-UZOjm4YdjAyrdHNcNEpnnsWp4b5x9PVG089D0bqb5qJwZog70BpYJq1VeF43BNZxZTV85aiyMBhF5XtVZ2NRzV8IL6lsDEyfdtyRyiN1TuyjxxN9cv3DN2d00CFnabIHlnTFMC8tGigDm-BPugApqp_D5D4vqjPqc0yuu7ispJ_kHWx7S0xoYfc2d3fe6-GXwjAc7eSQigSMvSnkyJTRCQrwllvZT6FtWnGxqsxrWpQVrkhXymtxrrqaZzOQsbbabvDdvuII6Z4NLvuviCK8gyHHG8khZxQA_R1063nEgvqcDXmpmVdccLHK4jpJ84lot7P-S7YEoU_r1l6AHV6J6diTKXrLLRgQ4gwqqsxosMy3ez5CILGAyGiGz2LheCwckANXh3AShxhBd0KxykUyICfllGqTQZ8ZdH9nuJFJhf-cy60tij0vvk8fK0AyeRqG-_14iRrDVhsm6URjYoBpfX3Ouk6ZpG-C0PRIQmIXAnanVFTcmSPARFAg2eyOoaWrCZJ5hLrN8YHJK5pJrsIceeRD6V_OOgpNUg70j19Zj5xlSrgXQbNQrMbQ1DeaSmMgVrDj-31uSPfXBCVdGPRBhKyxqf6rtKKHDrWLBkroE7PDQnZ8MRUUJe_HjnUiuQ)

In [2]:
from jupyterirtools import athena
from datetime import date
from datetime import timedelta

import importlib
importlib.reload(athena)

today = date.today()
yesterday = today - timedelta(days = 1)
    
sql = f"""SELECT srcaddr, srcport, dstaddr, dstport, count(packets) num_packets, sum(packets) packet_count, sum(bytes) sum_bytes
FROM "${{database_name}}"."vpc_flow_logs" 
WHERE "timestamp" >= '{str(yesterday.year).zfill(2)}/{str(yesterday.month).zfill(2)}/{str(yesterday.day).zfill(2)}'
AND accountid = '{account_id}' AND interface_id = '{eni_id}'
GROUP BY interface_id, srcaddr, srcport, dstaddr, dstport
ORDER BY sum(bytes) DESC
limit 1000;"""

#print(sql)
display(athena.run_query(sql))

""


# Recommended Actions
* Determine if IPs are internal or external.
* Perform a reputation lookup for the most frequently accessed external IPs. 
* Is a WAF rule needed to block the IP, or adjustments to the rate throttles.